<a href="https://colab.research.google.com/github/NRek95/DS_2024/blob/main/DS_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, classification_report
import os
import numpy as np
import gc

#from google.colab import drive
#os.chdir('C:/Users/t.boschert/OneDrive/Master/Data Science/04_Instacart_Data_Set/Instacart')
from google.colab import drive
drive.mount('/content/drive')
shared_folder_path = '/content/drive/My Drive/Group_DS'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
orders = pd.read_csv(shared_folder_path + '/orders.csv')
tip_train = pd.read_csv(shared_folder_path + '/tip_trainingsdaten1_.csv')
order_products_prior = pd.read_csv(shared_folder_path + '/order_products__prior.csv')
products = pd.read_csv(shared_folder_path + '/products.csv')

# Löschen der nicht-nemerischen Spalten
orders = orders.drop(columns=['eval_set'])
tip_train = tip_train.drop(columns=['Unnamed: 0'])
products = products.drop(columns=['product_name'])

# Ändern der Werte in der Spalte "tip" von True/False auf 1/0
tip_train['tip'] = tip_train['tip'].astype(int)

#instacart ist ein Unternehmen vergleichbar mit Gorillas

In [ ]:
#Sampling
users = pd.DataFrame(orders.user_id.unique())
users.columns=['user_id']

users_sample = users.sample(frac=0.3, random_state=0)
orders_sample = pd.merge(orders,users_sample)

In [ ]:
orders.head()

,order_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,1,2,8,NaN
1,2398795,1,2,3,7,15.0
2,473747,1,3,3,12,21.0
3,2254736,1,4,4,7,29.0
4,431534,1,5,4,15,28.0


In [ ]:
#mit neuem Vorgehen werden nur 44507 User miteinbezogen. 30% von 148143 wären allerdings 44442. Die geringe Differenz kommt vermutlich durch das Mergen zustande
df = pd.merge(orders_sample, order_products_prior, how='inner', on='order_id')
df = pd.merge(df, products, how='inner', on='product_id')
df = pd.merge(df, tip_train, how='inner', on='order_id')
df

,order_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,aisle_id,department_id,tip
0,2539329,1,1,2,8,NaN,196,1,0,77,7,0
1,2539329,1,1,2,8,NaN,14084,2,0,91,16,0
2,2539329,1,1,2,8,NaN,12427,3,0,23,19,0
3,2539329,1,1,2,8,NaN,26088,4,0,23,19,0
4,2539329,1,1,2,8,NaN,26405,5,0,54,17,0
...,...,...,...,...,...,...,...,...,...,...,...,...
9671736,2031003,180904,13,1,17,1.0,37758,1,1,124,5,1
9671737,3058118,184549,7,3,19,1.0,26121,1,0,109,11,0
9671738,2614894,187757,11,3,11,6.0,44978,1,0,40,8,0
9671739,1530060,194552,5,6,16,30.0,34085,1,1,27,5,0


In [ ]:
del orders
del tip_train
del order_products_prior
del products

In [ ]:
gc.collect()

0

# Data Analysis

In [ ]:
max_order_numbers = df.groupby('user_id')['order_number'].max().reset_index()
df_last_order = pd.merge(df, max_order_numbers, on=['user_id', 'order_number'])

In [ ]:
max_order_numbers

,user_id,order_number
0,1,10
1,6,3
2,13,12
3,15,22
4,17,40
...,...,...
61858,206200,23
61859,206202,22
61860,206204,4
61861,206205,3


In [ ]:
df_last_order

,order_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,aisle_id,department_id,tip
0,2550362,1,10,4,8,30.0,196,1,1,77,7,0
1,2550362,1,10,4,8,30.0,12427,9,1,23,19,0
2,2550362,1,10,4,8,30.0,10258,6,1,117,19,0
3,2550362,1,10,4,8,30.0,13032,8,1,121,14,0
4,2550362,1,10,4,8,30.0,25133,5,1,21,16,0
...,...,...,...,...,...,...,...,...,...,...,...,...
639404,3166965,107280,21,4,10,30.0,35701,1,1,26,7,0
639405,2882851,118406,9,6,15,30.0,21264,1,0,100,21,0
639406,693552,189086,3,0,16,30.0,31353,1,0,100,21,1
639407,1387947,152637,9,6,7,14.0,40755,1,1,92,18,1


In [ ]:
max_order_numbers2 = df.groupby('user_id')['order_number'].transform(max)

# Filtern der Zeilen, die nicht die maximalen order_number sind
filtered_df = df[df['order_number'] < max_order_numbers2]

In [ ]:
#df_wo_last_order = df_common[df_common['_merge'] == 'left_only'].drop(columns=['_merge'])

In [ ]:
max_order_numbers2

0          10
1          10
2          10
3          10
4          10
           ..
9671736    24
9671737    21
9671738    35
9671739     5
9671740    41
Name: order_number, Length: 9671741, dtype: int64

In [ ]:
filtered_df

,order_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,aisle_id,department_id,tip
0,2539329,1,1,2,8,NaN,196,1,0,77,7,0
1,2539329,1,1,2,8,NaN,14084,2,0,91,16,0
2,2539329,1,1,2,8,NaN,12427,3,0,23,19,0
3,2539329,1,1,2,8,NaN,26088,4,0,23,19,0
4,2539329,1,1,2,8,NaN,26405,5,0,54,17,0
...,...,...,...,...,...,...,...,...,...,...,...,...
9671735,724905,174363,12,4,23,30.0,32847,1,0,65,11,1
9671736,2031003,180904,13,1,17,1.0,37758,1,1,124,5,1
9671737,3058118,184549,7,3,19,1.0,26121,1,0,109,11,0
9671738,2614894,187757,11,3,11,6.0,44978,1,0,40,8,0


In [ ]:
del df
gc.collect()

17

# Feature Engineering


## user-specific features

In [ ]:
#mean basket size
mean_basket_size = df.groupby('user_id')['add_to_cart_order'].mean()
mean_basket_size
df = df.merge(mean_basket_size, on='user_id', how='inner')
df

KeyError: 'Column not found: add_to_cart_order'

In [ ]:
# Nutzeraktivität
user_activity = df.groupby('user_id').agg({
    'order_number': 'max',
    'days_since_prior_order': ['max', 'std']
}).reset_index()
user_activity.columns = ['user_id', 'total_orders', 'max_days_since_prior', 'std_days_since_prior']
user_activity.head()

,user_id,total_orders,max_days_since_prior,std_days_since_prior
0,1,10,30.0,9.304463
1,6,3,12.0,2.898275
2,13,12,13.0,1.727944
3,15,22,21.0,4.815927
4,17,40,30.0,7.451242


## product-specific features

In [ ]:
# Produktinformationen
product_info = df.groupby('product_id').agg({
    'reordered': 'mean',
    'add_to_cart_order': ['mean', 'std']
}).reset_index()
product_info.columns = ['product_id', 'reorder_rate', 'mean_add_to_cart_order', 'std_add_to_cart_order']
product_info.head()

,product_id,reorder_rate,mean_add_to_cart_order,std_add_to_cart_order
0,1,0.658805,5.317610,4.592332
1,2,0.290323,11.032258,8.068390
2,3,0.759494,6.468354,7.353414
3,4,0.543478,8.456522,5.987924
4,5,0.500000,8.000000,5.656854


# time-specific features

In [ ]:
# Zeitliche Merkmale
df['order_dow_hour'] = df['order_dow'].astype(str) + '_' + df['order_hour_of_day'].astype(str)
time_features = df.groupby(['user_id', 'order_dow_hour']).size().unstack(fill_value=0).reset_index()
times_features.head()

# Data Science

In [ ]:
# Nutzeraktivität
user_activity = df.groupby('user_id').agg({
    'order_number': 'max',
    'days_since_prior_order': ['mean', 'std']
}).reset_index()
user_activity.columns = ['user_id', 'total_orders', 'mean_days_since_prior', 'std_days_since_prior']

# Produktinformationen
product_info = df.groupby('product_id').agg({
    'reordered': 'mean',
    'add_to_cart_order': ['mean', 'std']
}).reset_index()
product_info.columns = ['product_id', 'reorder_rate', 'mean_add_to_cart_order', 'std_add_to_cart_order']

# Zeitliche Merkmale
df['order_dow_hour'] = df['order_dow'].astype(str) + '_' + df['order_hour_of_day'].astype(str)
time_features = df.groupby(['user_id', 'order_dow_hour']).size().unstack(fill_value=0).reset_index()

# Berechnen der Tip-Rate für jeden Benutzer
user_tip_rate = df.groupby('user_id')['tip'].mean().reset_index()
user_tip_rate.columns = ['user_id', 'avg_tip_rate']

# Zusammenführen der Features
data = df.merge(user_activity, on='user_id')
data = data.merge(product_info, on='product_id')
data = data.merge(time_features, on='user_id', how='left')
data = data.merge(user_tip_rate, on='user_id', how='left')

# Umwandeln von kategorischen Daten in numerische
data['order_dow_hour'] = data['order_dow_hour'].astype('category').cat.codes

# Feature-Matrix und Zielvariable
X = data.drop(columns=['tip', 'user_id', 'product_id', 'order_id'])
y = data['tip']

# Überprüfen, dass alle Spalten numerisch sind
print(X.dtypes)

# Train-Test-Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# XGBoost Modell definieren und trainieren
model = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
model.fit(X_train, y_train)

# Vorhersagen treffen
y_pred_proba = model.predict_proba(X_test)[:, 1]

# Modellbewertung
accuracy = accuracy_score(y_test, (y_pred_proba > 0.5).astype(int))
roc_auc = roc_auc_score(y_test, y_pred_proba)

print(f'Accuracy: {accuracy}')
print(f'ROC AUC: {roc_auc}')
print(classification_report(y_test, (y_pred_proba > 0.5).astype(int)))

# Wahrscheinlichkeiten für das gesamte Testset anzeigen
probabilities = pd.DataFrame({'Actual': y_test, 'Predicted Probability': y_pred_proba})
print(probabilities)


<h1>Analyzing the order_products dataset</h1>

---



In [ ]:
# Grundlegende Informationen und Statistiken
print(order_products_prior.info())
print(order_products_prior.describe())

# Überprüfung auf fehlende Werte
print(order_products_prior.isnull().sum())

# Häufigkeitsverteilung der 'reordered' Spalte
reordered_counts = order_products_prior['reordered'].value_counts()
print(reordered_counts)

# Visualisierung der Häufigkeitsverteilung der 'reordered' Spalte
plt.figure(figsize=(6,4))
sns.countplot(x='reordered', data=order_products_prior)
plt.title('Distribution of Reordered Products')
plt.xlabel('Reordered')
plt.ylabel('Count')
plt.show()

# Verteilung der Anzahl der Produkte pro Bestellung
products_per_order = order_products_prior.groupby('order_id').size()
print(products_per_order.describe())

plt.figure(figsize=(10,6))
sns.histplot(products_per_order, bins=50, kde=True)
plt.title('Distribution of Number of Products per Order')
plt.xlabel('Number of Products')
plt.ylabel('Frequency')
plt.show()

# Häufigste Produkte
top_products = order_products_prior['product_id'].value_counts().head(10)
print(top_products)

# Visualisierung der Top 10 Produkte
plt.figure(figsize=(12,8))
sns.barplot(x=top_products.index, y=top_products.values, palette='viridis')
plt.title('Top 10 Most Frequently Ordered Products')
plt.xlabel('Product ID')
plt.ylabel('Count')
plt.show()

# Heatmap der Korrelationen (wenn es numerische Features gibt)
correlation_matrix = order_products_prior.corr()
plt.figure(figsize=(8,6))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', linewidths=0.5)
plt.title('Correlation Matrix')
plt.show()


<h1>Analyzing the orders dataset</h1>

In [ ]:
# Grundlegende Informationen und Statistiken
print(orders.info())
print(orders.describe())

# Überprüfung auf fehlende Werte
print(orders.isnull().sum())

# Häufigkeitsverteilung der 'order_dow' Spalte
order_dow_counts = orders['order_dow'].value_counts()
print(order_dow_counts)

# Visualisierung der Häufigkeitsverteilung der 'order_dow' Spalte
plt.figure(figsize=(8,6))
sns.countplot(x='order_dow', data=orders)
plt.title('Distribution of Orders by Day of Week')
plt.xlabel('Day of Week')
plt.ylabel('Count')
plt.show()

# Verteilung der 'order_hour_of_day' Spalte
plt.figure(figsize=(10,6))
sns.histplot(orders['order_hour_of_day'], bins=24, kde=True)
plt.title('Distribution of Orders by Hour of Day')
plt.xlabel('Hour of Day')
plt.ylabel('Frequency')
plt.show()


In [ ]:
# Grundlegende Informationen und Statistiken
print(products.info())
print(products.describe())

# Überprüfung auf fehlende Werte
print(products.isnull().sum())

# Häufigkeitsverteilung der 'department_id' Spalte
department_counts = products['department_id'].value_counts()
print(department_counts)

# Visualisierung der Häufigkeitsverteilung der 'department_id' Spalte
plt.figure(figsize=(12,8))
sns.barplot(x=department_counts.index, y=department_counts.values, palette='viridis')
plt.title('Distribution of Products by Department')
plt.xlabel('Department ID')
plt.ylabel('Count')
plt.show()


# Data Preperation

In [ ]:
# Nutzeraktivität
user_activity = orders.groupby('user_id').agg({
    'order_number': 'max',
    'days_since_prior_order': ['mean', 'std']
}).reset_index()
user_activity.columns = ['user_id', 'total_orders', 'mean_days_since_prior', 'std_days_since_prior']

# Produktinformationen
product_info = orders.groupby('product_id').agg({
    'reordered': 'mean',
    'add_to_cart_order': ['mean', 'std']
}).reset_index()
product_info.columns = ['product_id', 'reorder_rate', 'mean_add_to_cart_order', 'std_add_to_cart_order']

# Zeitliche Merkmale
orders['order_dow_hour'] = orders['order_dow'].astype(str) + '_' + orders['order_hour_of_day'].astype(str)
time_features = orders.groupby(['user_id', 'order_dow_hour']).size().unstack(fill_value=0).reset_index()

In [ ]:
# Zusammenführen der Features
data = orders.merge(user_activity, on='user_id')
data = data.merge(product_info, on='product_id')
data = data.merge(time_features, on='user_id', how='left')

# Hinzufügen der Zielvariable
data = data.merge(tip_train, on='order_id', how='left')
data['tip'] = data['tip'].fillna(0)  # Annahme: fehlende Werte bedeuten kein Trinkgeld

# Umwandeln von kategorischen Daten in numerische
data['order_dow_hour'] = data['order_dow_hour'].astype('category').cat.codes

# Sicherstellen, dass alle Spalten numerisch sind
for column in data.columns:
    if data[column].dtype == 'object':
        data[column] = data[column].astype('category').cat.codes

# Fixed Train_Test_Split without random picking -> first 80% of data for train and last 20% for testing

In [ ]:
# Feature-Matrix und Zielvariable
X = data.drop(['tip', 'user_id', 'product_id', 'order_id'], axis=1)
y = data['tip']

# Überprüfen, dass alle Spalten numerisch sind
print(X.dtypes)

# Train-Test-Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0, shuffle=False)


<h1>Analyzing the correlation between the userid and the tip rate</h1>


In [ ]:
# Berechnen der Tip-Rate für jeden Benutzer
user_tip_rate = data.groupby('user_id')['tip'].mean().reset_index()
user_tip_rate.columns = ['user_id', 'avg_tip_rate']

# Zusammenführen der Tip-Rate mit den ursprünglichen Daten
data = data.merge(user_tip_rate, on='user_id', how='left')

# Überprüfen der Daten
print(orders_with_products[['user_id', 'order_id', 'tip', 'avg_tip_rate']].head())


# Data Science

In [ ]:
# DMatrix für XGBoost
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

# Parameter für XGBoost
params = {
    'objective': 'binary:logistic',
    'eval_metric': 'logloss',
    'eta': 0.1,
    'max_depth': 6
}

# Modelltraining
bst = xgb.train(params, dtrain, num_boost_round=100)

# Vorhersagen
y_pred = bst.predict(dtest)
y_pred_binary = [1 if pred > 0.5 else 0 for pred in y_pred]

# Evaluierung
accuracy = accuracy_score(y_test, y_pred_binary)
precision = precision_score(y_test, y_pred_binary)
recall = recall_score(y_test, y_pred_binary)
f1 = f1_score(y_test, y_pred_binary)

print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1-Score: {f1}')